# Stitch Slack Messages

This notebook demonstrates how to use the IndustryDocsSearch wrapper to query documents, process OCR content from a zip file, filter messages, and stitch conversations into a formatted text file.

<img src = "doc_screenshot.png" width = 900>

In [27]:
import os
import requests
import polars as pl
from industry_documents_wrapper.ucsf_api import IndustryDocsSearch

## Query Metadata Using IndustryDocsSearch

We're going to query the database using the Python wrapper for the UCSF Industry Documents API. The result of the query will provide the metadata, including document IDs, required to pull OCR text from the downloaded zip file.

In [99]:
wrapper = IndustryDocsSearch()
wrapper.query(q="type:'unknown' AND box:'DAWLDENGP'", n=-1)
wrapper.save('messages_results.json', format='json')

Adding URLs to query results


In [100]:
len(wrapper.results)

680

In [101]:
df = pl.read_json('messages_results.json')

In [102]:
print(set(df["box"]))

{'DAWLDENGP'}


In [59]:
df.head()

id,collection,collectioncode,box,availability,source,filepath,case,author,documentdate,type,pages,recipient,brand,bates,dateaddeducsf,attachment
str,list[str],list[str],str,list[str],str,list[str],list[str],list[str],str,list[str],i64,list[str],list[str],str,str,list[str]
"""flyc0324""","[""JUUL Labs Collection""]","[""juul""]","""DAWLDENGP""","[""public"", ""no restrictions""]","""[{""type"":""plaintext"",""title"":""…","[""Direct Messages\DAWLDENGP""]","[""State of North Carolina, ex rel. Joshua H. Stein, Attorney General, v. JUUL Labs, Inc""]","[""Ramean Behnam""]","""2019 January 14""","[""unknown""]",1,"[""Jonathan Powell"", ""Ramean Behnam""]","[""Juul""]","""JLI50358877""","""2024 September 26""",null
"""fmyc0324""","[""JUUL Labs Collection""]","[""juul""]","""DAWLDENGP""","[""public"", ""no restrictions""]","""[{""type"":""plaintext"",""title"":""…","[""Direct Messages\DAWLDENGP""]","[""State of North Carolina, ex rel. Joshua H. Stein, Attorney General, v. JUUL Labs, Inc""]","[""Ramean Behnam""]","""2019 January 21""","[""unknown""]",1,"[""Jonathan Powell"", ""Ramean Behnam""]","[""Juul""]","""JLI50358893""","""2024 September 26""",null
"""fpmh0327""","[""JUUL Labs Collection""]","[""juul""]","""DAWLDENGP""","[""public"", ""no restrictions""]","""[{""type"":""plaintext"",""title"":""…","[""Direct Messages\DAWLDENGP""]","[""State of North Carolina, ex rel. Joshua H. Stein, Attorney General, v. JUUL Labs, Inc""]","[""Ramean Behnam""]","""2018 November 30""","[""unknown""]",1,"[""Jonathan Powell"", ""Ramean Behnam""]","[""Juul""]","""JLI50132758""","""2024 September 26""",null
"""fqmh0327""","[""JUUL Labs Collection""]","[""juul""]","""DAWLDENGP""","[""public"", ""no restrictions""]","""[{""type"":""plaintext"",""title"":""…","[""Direct Messages\DAWLDENGP""]","[""State of North Carolina, ex rel. Joshua H. Stein, Attorney General, v. JUUL Labs, Inc""]","[""Ramean Behnam""]","""2018 December 05""","[""unknown""]",1,"[""Jonathan Powell"", ""Ramean Behnam""]","[""Juul""]","""JLI50132774""","""2024 September 26""",null
"""frmh0327""","[""JUUL Labs Collection""]","[""juul""]","""DAWLDENGP""","[""public"", ""no restrictions""]","""[{""type"":""plaintext"",""title"":""…","[""Direct Messages\DAWLDENGP""]","[""State of North Carolina, ex rel. Joshua H. Stein, Attorney General, v. JUUL Labs, Inc""]","[""Jonathan Powell""]","""2019 January 09""","[""unknown""]",1,"[""Jonathan Powell"", ""Ramean Behnam""]","[""Juul""]","""JLI50132790""","""2024 September 26""",null


## Extract OCR Content from the Zipped Folder
Now, we will extract the OCR content for the queried documents using the IDs retrieved in the previous step.

In [103]:
import zipfile

In [104]:
def get_ocr_content(df, zip_dir):
    id_list = df['id'].str.strip_chars().to_list()
    main_df = pl.DataFrame()

    with zipfile.ZipFile(zip_dir, 'r') as zipf:
        for idx, file in enumerate(zipf.namelist()):
            with zipf.open(file) as f:
                try:
                    temp_df = pl.read_csv(f, separator='|')
                    temp_df = temp_df.filter(pl.col('id').cast(pl.String).str.strip_chars().is_in(id_list))
                    main_df = pl.concat([main_df, temp_df])
                except Exception as e:
                    print(f'Error processing file {file}: {e}')
                    continue
            print(f'CSVs Checked: {idx+1}/{len(zipf.namelist())} | Docs Matched: {len(main_df)}/{len(id_list)}', end='\r')

    return main_df

In [105]:
zipdirectory = "../data/JUUL_Labs_Collection.zip"
df_text = get_ocr_content(df, zipdirectory)

Error processing file JUUL_Labs_Collection_56.csv: type String is incompatible with expected type Int64
Error processing file JUUL_Labs_Collection_172.csv: type String is incompatible with expected type Int64
Error processing file JUUL_Labs_Collection_195.csv: type String is incompatible with expected type Int64
Error processing file JUUL_Labs_Collection_197.csv: type String is incompatible with expected type Int64
Error processing file JUUL_Labs_Collection_214.csv: type String is incompatible with expected type Int64
Error processing file JUUL_Labs_Collection_296.csv: type String is incompatible with expected type Int64
Error processing file JUUL_Labs_Collection_515.csv: type String is incompatible with expected type Int64
Error processing file JUUL_Labs_Collection_717.csv: type String is incompatible with expected type Int64
Error processing file JUUL_Labs_Collection_778.csv: type String is incompatible with expected type Int64
Error processing file JUUL_Labs_Collection_1029.csv: typ

## Sort and Filter the Extracted Data
We will now sort the documents by their bates value and filter for the ones in the "Direct Messages" file path.

In [106]:
# order messages by bates
df_sorted = df_text.sort('bates')

In [184]:
# filter items with 'Direct Messages; in "File Path"

df_filtered = df_sorted.filter(
    pl.col('filepath') == 'Direct Messages\\DAWLDENGP'
)

In [185]:
df_filtered.head()

id,tid,bates,type,description,title,author,mentioned,attending,copied,recipient,redacted,collection_name,pages,exhibit_number,document_date,date_added_ucsf,date_modified_ucsf,date_added_industry,date_modified_industry,date_produced,date_shipped,deposition_date,date_privilege_logged,case,industry,drug,adverse_ruling,area,bates_alternate,box,brand,country,language,court,format,express_waiver,file,genre,keywords,bates_master,other_number,request_number,minnesota_request_number,privilege_code,topic,witness,cited,availability,grant_number,source,folder,series,chemical,food,rights,attachment,attachmentnum,conversation,conversationid,custodian,datereceived,datesent,filename,filepath,messageid,subject,timereceived,timesent,redaction,ocr_text
str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""knmh0327""",""" ""","""JLI50132731""","""unknown""",""" """,""" ""","""Ramean Behnam""",""" """,""" """,""" ""","""Jonathan Powell, Ramean Behnam""",""" ""","""JUUL Labs Collection""",1,""" ""","""Thu Sep 06 17:00:00 PDT 2018""","""Wed Sep 25 17:00:00 PDT 2024""","""Wed Sep 25 17:00:00 PDT 2024""",""" """,""" """,""" """,""" """,""" """,""" ""","""State of North Carolina, ex re…","""Tobacco""",""" """,""" """,""" """,""" ""","""DAWLDENGP""","""Juul""",""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" ""","""public, no restrictions""",""" ""","""{""type"":""plaintext"",""title"":""U…",""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" ""","""Direct Messages\DAWLDENGP""",""" """,""" """,""" """,""" """,""" ""","""Juul style CONFIDENTIAL NC…"
"""lnmh0327""",""" ""","""JLI50132732""","""unknown""",""" """,""" ""","""Jonathan Powell""",""" """,""" """,""" ""","""Jonathan Powell, Ramean Behnam""",""" ""","""JUUL Labs Collection""",1,""" ""","""Thu Sep 06 17:00:00 PDT 2018""","""Wed Sep 25 17:00:00 PDT 2024""","""Wed Sep 25 17:00:00 PDT 2024""",""" """,""" """,""" """,""" """,""" """,""" ""","""State of North Carolina, ex re…","""Tobacco""",""" """,""" """,""" """,""" ""","""DAWLDENGP""","""Juul""",""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" ""","""public, no restrictions""",""" ""","""{""type"":""plaintext"",""title"":""U…",""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" ""","""Direct Messages\DAWLDENGP""",""" """,""" """,""" """,""" """,""" ""","""Right? CONFIDENTIAL NC-JLI…"
"""mnmh0327""",""" ""","""JLI50132733""","""unknown""",""" """,""" ""","""Jonathan Powell""",""" """,""" """,""" ""","""Jonathan Powell, Ramean Behnam""",""" ""","""JUUL Labs Collection""",1,""" ""","""Thu Sep 06 17:00:00 PDT 2018""","""Wed Sep 25 17:00:00 PDT 2024""","""Wed Sep 25 17:00:00 PDT 2024""",""" """,""" """,""" """,""" """,""" """,""" ""","""State of North Carolina, ex re…","""Tobacco""",""" """,""" """,""" """,""" ""","""DAWLDENGP""","""Juul""",""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" ""","""public, no restrictions""",""" ""","""{""type"":""plaintext"",""title"":""U…",""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" """,""" ""","""Direct Messages\DAWLDENGP""",""" """,""" """,""" """,""" """,""" ""","""She said she'll be in the main…"
"""nnmh0327""",""" ""","""JLI50132734""","""unknown""",""" """,""" ""","""Ramean Behnam""",""" """,""" """,""" ""","""Jonathan Powell, Ramean Behnam""",""" ""","""JUUL Labs Collection""",1,""" ""","""Thu Sep 06 17:00:00 PDT 2018""","""Wed Sep 25 17:00:00 PDT 2024""","""Wed Sep 25 17:00:00 PDT 2024""",""" """,""" """,""" """,""" """,""

## Stitch Conversations
Here, we will format the extracted messages into a conversational format. We will also clean the content to remove confidential information and unnecessary metadata.

In [202]:
# Stitches the conversation within box: DAWLDENGP
import re

def stitch_conversation(df: pl.DataFrame):
    
    conversation = []
    for row in df.iter_rows(named=True):
        date = row.get('document_date')
        author = row.get('author')
        content = row.get('ocr_text')
        
        # clean the content to remove CONFIDENTIAL and extra numbers so it just shows the slack message
        cleaned_content = re.sub(r"CONFIDENTIAL\s+NC-JLI-Consent Judgment\s+JL\d+", "", content).strip()

        # format as Date - Author - Content
        conversation.append(f"{date} -- {author} -- {cleaned_content}\n")

    return "\n".join(conversation)


In [203]:
conversation = stitch_conversation(df_filtered)
print(conversation)

Thu Sep 06 17:00:00 PDT 2018 -- Ramean Behnam -- Juul style

Thu Sep 06 17:00:00 PDT 2018 -- Jonathan Powell -- Right?

Thu Sep 06 17:00:00 PDT 2018 -- Jonathan Powell -- She said she'll be in the main office in -20 min

Thu Sep 06 17:00:00 PDT 2018 -- Ramean Behnam -- I'm in London she is in SF

Thu Sep 06 17:00:00 PDT 2018 -- Jonathan Powell -- She's in London. Promise.

Sun Sep 09 17:00:00 PDT 2018 -- Ramean Behnam -- What's up man.. . . I'm trying to do age verification for my account but it doesn't want to accept my ID. says  failed to upload. any ideas?

Sun Sep 09 17:00:00 PDT 2018 -- Jonathan Powell -- N000! I've heard of other people having that problem. I don't know what causes it.    CONFIDENTIAL  NC-JLI-Consent Judgment    I'll Slack Eadon.    JL150132737

Sun Sep 09 17:00:00 PDT 2018 -- Ramean Behnam -- Thanks. It's a small file and everything lol

Sun Sep 09 17:00:00 PDT 2018 -- Jonathan Powell -- Are you able to proceed w/o uploading an image?    If you enter all your in

## Save the Conversation to a Text File
Finally, we save the stitched conversation into a text file for further analysis or archiving.

In [201]:
# save as text file
def save_conversation_to_file(df:pl.DataFrame, output_file: str):
    conversation = stitch_conversation(df)
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(conversation)

save_conversation_to_file(df_filtered, f"chat_{df_filtered['box'][0]}.txt")